# Amortized Inference for a NLME Model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pypesto import visualize, optimize, profile, engine

from inference.helper_functions import (create_mixed_effect_model_param_names,
                                        analyse_correlation_in_posterior,
                                        create_fixed_params)
from inference.inference_functions import run_population_optimization
from inference.nlme_objective import ObjectiveFunctionNLME
from inference.ploting_routines import (plot_real_vs_synthetic,
                                        plot_real_and_estimated,
                                        plot_estimated_distributions,
                                        visualize_pesto_result)

In [ ]:
# specify which model to use
model_name = ['fröhlich-simple', 'fröhlich-detailed', 'fröhlich-sde', 
              'pharmacokinetic_model', 
              'clairon_small_model'][-1]

# Load individual model

In [ ]:
if model_name == 'fröhlich-simple':
    from models.froehlich_model_simple import FroehlichModelSimple
    individual_model = FroehlichModelSimple(load_best=True)
elif model_name == 'fröhlich-detailed':
    from models.froehlich_model_detailed import FroehlichModelDetailed
    individual_model = FroehlichModelDetailed(load_best=True)
elif model_name == 'fröhlich-sde':
    from models.froehlich_model_sde import FroehlichModelSDE
    individual_model = FroehlichModelSDE(load_best=True)    
elif model_name == 'pharmacokinetic_model':
    from models.pharmacokinetic_model import PharmacokineticModel
    individual_model = PharmacokineticModel(load_best=True)    
elif model_name == 'clairon_small_model':
    from models.clairon_small_model import ClaironSmallModel
    individual_model = ClaironSmallModel(load_best=True)
else:
    raise NotImplementedError('model not implemented')

# assemble simulator and prior
trainer = individual_model.build_trainer('../networks/' + individual_model.network_name)
individual_model.plot_example()

## Load Data

In [ ]:
# define how many data points are used for optimization
n_data = 100
load_real_data = True
# load data
true_pop_parameters = None
results_to_compare = None
if 'Froehlich' in individual_model.name:
    obs_data = individual_model.load_data(n_data=n_data, load_egfp=load_real_data, load_d2egfp=False)
    if not load_real_data:
        true_pop_parameters = individual_model.load_synthetic_parameter(n_data=n_data)
    
    # load SDE data for comparison
    #from models.froehlich_model_sde import FroehlichModelSDE
    #model_sde = FroehlichModelSDE(load_best=True)
    #obs_data = model_sde.load_data(n_data=n_data, synthetic=True)
    #true_pop_parameters_sde = model_sde.load_synthetic_parameter(n_data=n_data)
else:
    obs_data = individual_model.load_data(n_data=n_data, synthetic=not load_real_data)

print(len(obs_data), 'individuals')

# Estimating Population Parameters

Now we want to use the amortizer to generate samples such that we can minimize the negative log-likelihood of the data given the population parameters of the mixed effect model:
$$
    \beta^*,\Psi^* \approx 
    \underset{\beta,\Psi}{\arg\min} -\sum_i \log\left( \frac1M \sum_j^M \frac{p(\phi_j \mid \beta,\Psi)}{p(\phi_j)} \right).
$$

Remark: the objective value is not the likelihood value since the sum over $\log p(y_i)$ is missing.


$\beta$ is called ```theta_population``` in the code.


$\log \phi$ cell specific parameters, sampled from $\mathcal{N}(\beta,\Psi)$
$$
    p( \phi \mid \beta,\Psi) = (2\pi)^{-k/2}\vert \Psi\vert^{-1/2} \prod_{l=1}^k \phi_l^{-1} \exp \left(-\frac12 (\log \phi-\beta)^T \Psi^{-1}  (\log \phi-\beta) \right)
$$

Assumptions to start with: $\Psi$ is a diagonal matrix, need better parameterization for other types

$$
    \beta^*,\Psi^* \approx
    \underset{\beta,\Psi}{\arg\min} -\sum_i \left(\log \frac1M \sum_j^M \frac{p( \phi_j \mid \beta,\Psi)}{p( \phi_j)} \right) \\
     =  \underset{\beta,\Psi}{\arg\min} -\sum_i \left(\log\left(\vert \Psi\vert^{-1/2} \right) -\log M -
    \log\left(\vert \Sigma\vert^{-1/2}\right) +\log \sum_j^M \exp \left(-\frac12 (\log\phi_j-\beta)^T \Psi^{-1}  (\log\phi_j-\beta) + \frac12 (\log\phi_j-\mu)^T \Sigma^{-1}  (\log\phi_j-\mu) \right)\right)
$$

if the prior is $p( \phi) = (2\pi)^{-k/2}\vert \Sigma\vert^{-1/2} \prod_{l=1}^k \phi_l^{-1}\exp \left(-\frac12 (\log \phi-\mu)^T \Sigma^{-1}  (\log\phi-\mu) \right)$.


For purpose of optimization we also parametrize $\Psi$ by a log-transformation since diagonal entries must be positive.

## Define Objective Function

- you have to choose the covariance format (diag or cholesky)
- if you want to use covariates, you have to specify the covariate mapping to the parameters of the log-normal distribution
- a covariate mapping takes in parameter samples (n_indv, n_samples, n_params), covariates (n_indv, n_covariates) and parameter vector with parameters needed to map the covariates into the other parameter from the mixed effects model and returns transformed parameter samples (n_indv, n_samples, n_params)

In [ ]:
cov_type = ['diag', 'cholesky'][1]
use_covariates = True

In [ ]:
# build covariate mapping if needed
covariates_bounds = None
covariates_names = []
if 'clairon' in model_name and use_covariates:
    covariates_names = ['c_age', 'c_gender']
    covariates_bounds = np.array([[-1, 1], [-1, 1]])

In [ ]:
if use_covariates and 'clairon' in model_name:
    _, covariates = individual_model.load_data(n_data=n_data, load_covariates=True)
    
    def additive_covariate_mapping(param_samples: np.ndarray, 
                                   covariates: np.ndarray, 
                                   covariate_params: np.ndarray, 
                                   threshold: float = 0.001):
        # add param_of_cov*covariates to parameters
        additive_cov = np.sum(covariate_params*covariates, axis=1)
        transformed_param_samples = np.exp(param_samples)
        
        for s_id in range(transformed_param_samples.shape[0]):
            # estimating exp(pop_mean), param_cov, variance of random_effect with 0 mean
            # exp(individual_param_i) = exp(pop_mean + random_effect) + param_cov * cov_i
            transformed_param_samples[s_id, :, :] -= additive_cov[s_id]  # cov on one parameter
        # if parameters are negative, set to threshold
        transformed_param_samples[transformed_param_samples < 0] = threshold
        param_samples_cov = np.log(transformed_param_samples)
        return param_samples_cov
    
    def multiplicative_covariate_mapping(param_samples: np.ndarray, 
                                         covariates: np.ndarray, 
                                         covariate_params: np.ndarray):
        # add param_of_cov*covariates to parameters
        additive_cov = np.sum(covariate_params*covariates, axis=1)
        
        for s_id in range(param_samples.shape[0]):
            # estimating exp(pop_mean), param_cov, variance of random_effect with 0 mean
            # individual_param_i = pop_mean + random_effect + param_cov * cov_i
            # since parameters are log-normal distributed, results in a multiplicative effect
            param_samples[s_id, :, :] -= additive_cov[s_id] 
        return param_samples
    
    covariate_mapping = [additive_covariate_mapping, multiplicative_covariate_mapping][0]
else:
    covariate_mapping = None
    covariates = None

obj_fun_amortized = ObjectiveFunctionNLME(model_name=individual_model.name,
                                          param_samples=np.empty((1,1,1)),
                                          prior_mean=individual_model.prior_mean,
                                          prior_std=individual_model.prior_std,
                                          covariance_format=cov_type,
                                          covariates=covariates,
                                          covariate_mapping=covariate_mapping,
                                          prior_type=individual_model.prior_type,
                                          prior_bounds=individual_model.prior_bounds if hasattr(individual_model, 'prior_bounds') else None,  # for uniform prior
                                          )

In [ ]:
mixed_effect_params_names = create_mixed_effect_model_param_names(individual_model.param_names, 
                                                                  cov_type=cov_type,
                                                                  covariates_names=covariates_names)
print(mixed_effect_params_names)

### Analyse correlations between parameters

In [ ]:
high_correlation_pairs = analyse_correlation_in_posterior(model=individual_model, 
                                                          mixed_effect_params_names=mixed_effect_params_names, 
                                                          obs_data=obs_data,
                                                          threshold_corr=0.5)
print('Parameter pairs of high correlation in individual posterior:', high_correlation_pairs)

## Fixed and Random Effects

Decide which parameters to fix
- a fixed effect is modeled as a random effect with variance 0 (all parameters follow a normal distribution)
- variance of error parameters in the individual model are usually supposed to be a fixed parameter in the population model
- correlations with these error parameters are usually fixed to 0


In [ ]:
if 'Froehlich' in individual_model.name:
    # fix variance of error parameters and correlations with sigma if cholesky covariance is used
    fix_names = ['var-$\sigma$'] + [name for name in mixed_effect_params_names if '\sigma' in name and 'corr_' in name]
    fixed_values = [0] * len(fix_names)
elif 'pharmacokinetic' in individual_model.name:
    fix_error_var = ['var-$\\theta_{12}$', 'var-$\\theta_{13}$']
    fix_error_var_val = [0] * len(fix_error_var)
    # fix variance of fixed parameters
    fixed_effects_var = ['var-$\\theta_1$', 'var-$\\theta_5$', 'var-$\\theta_7$', 'var-$\\theta_8$', 
                         'var-$\\theta_{10}$', 'var-$\\theta_{12}$', 'var-$\\theta_{13}$']
    fixed_effects_var_val = [0] * len(fixed_effects_var)
    # fix mean of random effect
    random_effect_mean = ['pop-$\eta_4$']
    random_effect_mean_val = [0]
    
    # put all fixed parameters together
    fix_names = fix_error_var + fixed_effects_var + random_effect_mean
    fixed_values = fix_error_var_val + fixed_effects_var_val + random_effect_mean_val
    
    # if correlations are used, only allow the same as in the original model
    # hence correlations with the error parameter are fixed as well
    if cov_type == 'cholesky':
        non_fix_corr = ['corr_$\\theta_2-\\eta_1$_$\\theta_6-\\eta_2$', 
                        'corr_$\\theta_4-\\eta_3$_$\\theta_6-\\eta_2$', 
                        'corr_$\\theta_4-\\eta_3$_$\\eta_4$']
        fixed_corr = [x for x in mixed_effect_params_names if 'corr_' in x and x not in non_fix_corr]
        fix_names += fixed_corr
        fixed_values += [0] * len(fixed_corr)
    
elif 'Clairon' in individual_model.name:
    fix_names = ['var-error_prop'] + [name for name in mixed_effect_params_names if 'error' in name and 'corr_' in name]
    fixed_values = [0] * len(fix_names)
else:
    raise NotImplementedError('model not yet implemented')
    
fixed_indices, fixed_values = create_fixed_params(fix_names=fix_names, 
                                                  fixed_values=fixed_values,
                                                  params_list=mixed_effect_params_names, 
                                                  fix_low_correlation=False,  # only applies to cholesky covariance
                                                  high_correlation_pairs=high_correlation_pairs)
print(mixed_effect_params_names)
# note: inf values in fixed_values will be set to upper or lower bound respectively

# Run Population Optimization

In [ ]:
%%time
result_optimization = run_population_optimization(
    individual_model=individual_model,
    data=obs_data,
    param_names=mixed_effect_params_names,
    objective_function=obj_fun_amortized,
    n_multi_starts=20,
    n_samples_opt=50,
    covariates_bounds=covariates_bounds,
    x_fixed_indices=fixed_indices,
    x_fixed_vals=fixed_values,
    file_name=None,
    verbose=True,
    trace_record=True,
    pesto_multi_processes=10,
    result=None #result_optimization
    )

results = result_optimization.optimize_result.as_dataframe()['x']

In [ ]:
#from pypesto.store import OptimizationResultHDF5Writer
#writer = OptimizationResultHDF5Writer(f'output/{model_name}_real_{n_data}_3.hdf5')
#writer.write(result_optimization)

In [ ]:
#from pypesto.store import OptimizationResultHDF5Reader
#result_optimization2 = OptimizationResultHDF5Reader(f'output/scalability/{model.name}_cells_{n_data}_samples_{50}.hd5').read()
#results = result_optimization.optimize_result.as_dataframe()['x']

In [ ]:
visualize_pesto_result(result_optimization)

In [ ]:
# show best result
estimated_beta = results[0][:individual_model.n_params]
var = results[0][individual_model.n_params:individual_model.n_params*2]
estimated_psi = obj_fun_amortized.get_covariance(results[0][individual_model.n_params:])

display(pd.DataFrame(estimated_beta,
                     index=mixed_effect_params_names[:individual_model.n_params],
                     columns=['estimated population parameters']))
display(pd.DataFrame(np.exp(-var),
                     index=mixed_effect_params_names[individual_model.n_params:individual_model.n_params*2],
                     columns=['estimated population parameters']))
if cov_type == 'cholesky' or use_covariates:
    display(pd.DataFrame(results[0][individual_model.n_params:],
                         index=mixed_effect_params_names[individual_model.n_params:],
                         columns=['estimated population parameters']))
    
display(pd.DataFrame(np.var([r for r in results], axis=0),
                     index=mixed_effect_params_names,
                     columns=['variance of multi-start results']))

In [ ]:
if true_pop_parameters is not None:
    print('true values of log-normal distribution')
    df_param_sample = pd.DataFrame(true_pop_parameters[np.newaxis,:].round(4),
                               columns=mixed_effect_params_names)
    display(df_param_sample)

if results_to_compare is not None:
    print('values of baseline method of log-normal distribution')
    df_param_sample = pd.DataFrame(results_to_compare, columns=mixed_effect_params_names)
    display(df_param_sample)

In [ ]:
estimated_psi_inv = obj_fun_amortized.get_inverse_covariance(results[0][individual_model.n_params:])

In [ ]:
from scipy.special import logsumexp

In [ ]:
n_samples_test = 1000
n_patients = 10
test_eval = np.zeros((n_patients, n_samples_test))

for i in range(n_patients):
    test_samples = individual_model.draw_posterior_samples(data=obs_data, n_samples=n_samples_test)[i]
    test_function = lambda x: -0.5*(x-estimated_beta).dot(estimated_psi_inv).dot(x-estimated_beta)
    test_eval[i] = np.array([test_function(x) for x in test_samples])
    
    
test_expectation = np.exp(test_eval).sum(axis=1) / n_samples_test
test_variance = 1/(n_samples_test-1) * np.sum((np.exp(test_eval) - test_expectation[:, np.newaxis])**2, axis=1)
test_error = np.sqrt(test_variance/n_samples_test)
print(f'E={test_expectation}')
print(f'Var={test_variance}')
print(f'Error={test_error}')
   
print('\n')
print('log-sum-exp', np.log(test_expectation))
print('logsumexp', -np.log(n_samples_test)+logsumexp(test_eval, axis=1))
print('sumlogsumexp', np.sum(-np.log(n_samples_test)+logsumexp(test_eval, axis=1)))
print('error-order', (1/(n_samples_test-1)))

In [ ]:
np.random.seed(42)
from models.base_nlme_model import batch_gaussian_prior
synthetic_params = batch_gaussian_prior(mean=model.prior_mean-1, cov=model.prior_cov/2, batch_size=n_data) - 1
synthetic_params[:, -2:] = model.prior_mean[-2:]-1
true_pop_parameters = np.concatenate((np.mean(synthetic_params, axis=0), np.var(synthetic_params, axis=0)))
display(true_pop_parameters)

In [ ]:
display((estimated_beta-true_pop_parameters[:model.n_params])**2 / true_pop_parameters[:model.n_params]**2)

In [ ]:
display((estimated_psi.diagonal()-true_pop_parameters[model.n_params:])**2 / true_pop_parameters[model.n_params:]**2)

In [ ]:
simulator = model.build_simulator(with_noise=True)

In [ ]:
if 'fröhlich' in model_name:
    plot_real_vs_synthetic(estimated_mean=estimated_beta,
                           estimated_cov=estimated_psi,
                           data=obs_data,
                           model_name=model.name,
                           n_trajectories=len(obs_data),
                           simulator=simulator,
                           #save_fig='dif_ode', #model_name+'_eGFP_dif' if load_real_data else model_name+'_synthetic_dif',
                           #estimation_function=np.median,
                           ylim=(-1.,1.),
                           seed=0)
    plot_real_and_estimated(estimated_mean=estimated_beta,
                           estimated_cov=estimated_psi,
                           data=obs_data,
                           model_name=model.name,
                           n_trajectories=len(obs_data),
                           simulator=simulator,
                           #save_fig=model_name+'_eGFP_estimate' if load_real_data else model_name+'_synthetic_estimate',
                           seed=0)

In [ ]:
def plot_parameter_estimates(result_list: list[np.ndarray],
                             param_names_plot: list[str],
                             prior_mean: np.ndarray = None,
                             prior_std: np.ndarray = None,
                             true_parameters: np.ndarray = None,
                             run_names: list[str] = None,  # None, if multi-starts are compared
                             save_fig: bool = False) -> None:
    # plot parameters
    fig, ax = plt.subplots(figsize=(15, 5))  # , dpi=600)
    parameters_ind = list(range(1, result_list[0].shape[0] + 1))[::-1]

    if prior_mean is not None and prior_std is not None:
        n_pop_params = len(prior_mean)
        prior_interval = np.array([prior_mean - 1.96 * prior_std, prior_mean + 1.96 * prior_std]).T
        ax.fill_betweenx(parameters_ind[:n_pop_params], prior_interval[:, 0], prior_interval[:, 1],
                         color='grey', alpha=0.2, label='95% prior region')

    for j_x, x in reversed(list(enumerate(result_list))):
        if run_names is None:
            if j_x == 0:
                pass
                #tmp_legend = 'optimal run'
            else:
                tmp_legend = None
        else:
            tmp_legend = run_names[j_x]
        ax.plot(
            x,
            parameters_ind,
            linestyle='dashed',
            color='blue',
            marker='o',
            label=tmp_legend,
        )
        
    if true_parameters is not None:
        ax.plot(true_parameters, parameters_ind, color='red', marker='x',
                label='true parameters sample')
        
    ax.set_yticks(parameters_ind, param_names_plot)
    ax.set_xlabel('Parameter value')
    ax.set_ylabel('Parameter')
    ax.set_title('Estimated Population Parameters (log-normal distribution)')
    ax.legend(loc=2, bbox_to_anchor=(1, 1))
    fig.tight_layout()
    return

In [ ]:
plot_parameter_estimates(results_transformed,
                         param_names_plot=full_param_names,
                         prior_mean=prior_mean,
                         prior_std=prior_std,
                         true_parameters=true_pop_parameters,
                         save_fig=False)

In [ ]:
plot_estimated_distributions(results_transformed[0],
                             param_names_plot=param_names,
                             prior_mean=prior_mean,
                             prior_std=prior_std,
                             true_parameters=true_pop_parameters,
                             save_fig=False)

In [ ]:
if results_to_compare is not None:
    compare_list = [results_transformed[0], results_to_compare[0]]
    plot_parameter_estimates(compare_list, param_names_plot=full_param_names,
                         prior_mean=model.prior_mean,
                         prior_std=model.prior_std,
                         true_parameters=true_pop_parameters,
                         run_names=['BayesFlow', 'Baseline'],
                         save_fig=False)

    print('Baseline Estimated Parameters')
    plot_parameter_estimates(results_to_compare, param_names_plot=full_param_names,
                         prior_mean=model.prior_mean,
                         prior_std=model.prior_std,
                         save_fig=False)

# Uncertainty Analysis

Uncertainty based on profiles -> more accurate

In [ ]:
profile_options = profile.ProfileOptions(
    min_step_size=0.0005, #0.001
    step_size_factor=1.1, #1.25
    delta_ratio_max=0.05, #0.1
    default_step_size=0.005, #0.01
    ratio_min=0.01, #0.145
)

result_optimization = profile.parameter_profile(
    problem=result_optimization.problem,
    result=result_optimization,
    optimizer=optimize.ScipyOptimizer(),
    engine=engine.MultiProcessEngine(10),
    #profile_index=np.array([0]),
    #result_index=0, # index from which optimization result profiling should be started
    #profile_options=profile_options,
    #filename=f'output/uncertainty/{model.name}_cells_{n_data}_samples_{50}.hd5',
    #overwrite=True
)

In [ ]:
import matplotlib

matplotlib.rcdefaults()  # for resetting to defaults
plt.rcParams.update({'font.size': 14,
                     #'text.usetex': True,
                     "font.family": "serif",
                     "font.serif": ["Computer Modern Roman"],
                     'axes.titlesize': 'small',
                     'axes.labelsize': 'small',
                     'xtick.labelsize': 'xx-small', # todo: change back to small
                     'ytick.labelsize': 'small',
                     'legend.fontsize': 'small',
                     #'figure.dpi': 600,
                     'figure.figsize': (16,12)}) #
colors = ['#1f78b4', '#a6cee3', '#b2df8a','#33a02c','#fb9a99']

In [ ]:
ax = visualize.profiles(result_optimization, size=(16,12))
#plt.savefig('plots/synthetic_profiles_small_stochastic_data.pdf', bbox_inches='tight', dpi=600, format='pdf')
plt.show()

In [ ]:
ax = visualize.profile_cis(result_optimization) #,
                                  #profile_indices=[0,1,2,3,4,5,6,7,8,9,10,11])
ax.set_title('Approximate Confidence Intervals \n Based on Profiles')
plt.tight_layout()
#plt.savefig('plots/synthetic_FIM_profiles_cis.png')
plt.show()

Uncertainty based on FIM

In [ ]:
result_optimization = profile.approximate_parameter_profile(
    problem=result_optimization.problem,
    result=result_optimization,
    #profile_index=np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    #result_index=0, # index from which optimization result profiling should be started
)

In [ ]:
visualize.profiles(result_optimization)
#plt.savefig('plots/synthetic_FIM_profiles.png')
plt.show()

In [ ]:
n_indv = obs_data.shape[0]
confidence_ratio = profile.chi2_quantile_to_ratio(alpha=0.95, df=n_indv-(model.n_params*2))

confidence_intervals = np.zeros((model.n_params*2, 2))

for param_idx in range(model.n_params*2):
    try:
        xs = result_optimization.profile_result.list[0][param_idx]['x_path'][param_idx]
        ratios = result_optimization.profile_result.list[0][0]['ratio_path']
        confidence_intervals[param_idx] = profile.calculate_approximate_ci(xs, ratios,
                                                                            confidence_ratio=confidence_ratio)
    except TypeError:
        print(f'for {full_param_names[param_idx]} confidence interval could not be estimated')

confidence_intervals